# MET Prediction Model Selection and Analysis

This notebook addresses the key questions for ADAMMA Challenge 2025:
1. **Time Window Optimization**: Analyzing whether momentary acceleration spikes should trigger immediate class changes or if we need smoothing
2. **Model Comparison**: Comparing multiple ML models for MET class prediction
3. **Mobile Deployment**: Preparing the best model for iOS app integration

## Key Research Questions:
- Should momentary acceleration spikes trigger activity class changes?
- What's the optimal time window for prediction?
- Which ML model provides the best accuracy-performance trade-off for mobile deployment?

In [ ]:
# Import Required Libraries and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from scipy import stats
from scipy.signal import butter, filtfilt
import warnings
import time
import os
import sys

# Add project paths
sys.path.append('../src')
from data_processing.wisdm_processor import WISDMDataProcessor
from feature_extraction.feature_extractor import METFeatureExtractor

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')

print("Libraries imported successfully!")
print(f"Working directory: {os.getcwd()}")

In [ ]:
# Load and Prepare Dataset
print("Loading WISDM dataset...")

# Initialize data processor
processor = WISDMDataProcessor(data_dir="../data")

# Load processed data or create synthetic if needed
try:
    df = pd.read_csv("../data/processed/synthetic_met_data.csv")
    print(f"Loaded existing dataset: {df.shape}")
except FileNotFoundError:
    print("Creating new dataset...")
    df = processor.load_and_process_data()

# Display dataset info
print(f"\nDataset Overview:")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nMET class distribution:")
print(df['met_class'].value_counts().sort_index())
print(f"\nActivity distribution:")
print(df['activity'].value_counts())

# Check for missing values
print(f"\nMissing values: {df.isnull().sum().sum()}")

In [ ]:
# Data Preprocessing and Basic Feature Engineering
def calculate_magnitude(x, y, z):
    """Calculate acceleration magnitude"""
    return np.sqrt(x**2 + y**2 + z**2)

def calculate_jerk(acc_data):
    """Calculate jerk (rate of change of acceleration)"""
    return np.diff(acc_data, prepend=acc_data[0])

def apply_smoothing_filter(data, cutoff_freq=5, fs=20, order=4):
    """Apply low-pass Butterworth filter to smooth data"""
    nyquist = 0.5 * fs
    normal_cutoff = cutoff_freq / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data)

# Calculate basic features
df['magnitude'] = calculate_magnitude(df['x'], df['y'], df['z'])
df['magnitude_smoothed'] = apply_smoothing_filter(df['magnitude'].values)

# Calculate statistical features per user-activity session
print("Calculating advanced features...")

def extract_features_for_window(group, window_size=50):
    """Extract features for a sliding window"""
    features = []
    for i in range(0, len(group) - window_size + 1, window_size//2):  # 50% overlap
        window = group.iloc[i:i+window_size]
        
        feature_row = {
            'user': window['user'].iloc[0],
            'activity': window['activity'].iloc[0],
            'met_class': window['met_class'].iloc[0],
            
            # Magnitude features
            'mag_mean': window['magnitude'].mean(),
            'mag_std': window['magnitude'].std(),
            'mag_min': window['magnitude'].min(),
            'mag_max': window['magnitude'].max(),
            'mag_range': window['magnitude'].max() - window['magnitude'].min(),
            'mag_iqr': window['magnitude'].quantile(0.75) - window['magnitude'].quantile(0.25),
            
            # Smoothed magnitude features  
            'mag_smooth_mean': window['magnitude_smoothed'].mean(),
            'mag_smooth_std': window['magnitude_smoothed'].std(),
            
            # Individual axis features
            'x_mean': window['x'].mean(),
            'x_std': window['x'].std(),
            'y_mean': window['y'].mean(), 
            'y_std': window['y'].std(),
            'z_mean': window['z'].mean(),
            'z_std': window['z'].std(),
            
            # Cross-axis correlations
            'xy_corr': window['x'].corr(window['y']),
            'xz_corr': window['x'].corr(window['z']),
            'yz_corr': window['y'].corr(window['z']),
            
            # Zero crossing rates
            'x_zcr': np.sum(np.diff(np.signbit(window['x']))) / len(window),
            'y_zcr': np.sum(np.diff(np.signbit(window['y']))) / len(window),
            'z_zcr': np.sum(np.diff(np.signbit(window['z']))) / len(window),
        }
        features.append(feature_row)
    
    return features

# Extract features for different window sizes
window_sizes = [25, 50, 100]  # Corresponding to ~1.25s, 2.5s, 5s at 20Hz
feature_datasets = {}

for window_size in window_sizes:
    print(f"Extracting features for window size: {window_size}")
    all_features = []
    
    for (user, activity), group in df.groupby(['user', 'activity']):
        if len(group) >= window_size:
            features = extract_features_for_window(group, window_size)
            all_features.extend(features)
    
    feature_df = pd.DataFrame(all_features)
    feature_df = feature_df.dropna()  # Remove rows with NaN correlations
    feature_datasets[window_size] = feature_df
    
    print(f"  - Generated {len(feature_df)} feature windows")
    print(f"  - MET class distribution: {feature_df['met_class'].value_counts().to_dict()}")

print("Feature extraction complete!")

In [ ]:
# Time Window Analysis and Threshold Investigation
print("=== TIME WINDOW ANALYSIS ===")

# Analyze the effect of different window sizes on classification stability
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Magnitude variance across different window sizes
for i, window_size in enumerate(window_sizes):
    feature_df = feature_datasets[window_size]
    
    # Plot magnitude statistics by MET class
    ax = axes[i//2, i%2] if len(window_sizes) > 2 else axes[i]
    
    for met_class in sorted(feature_df['met_class'].unique()):
        class_data = feature_df[feature_df['met_class'] == met_class]
        ax.scatter(class_data['mag_mean'], class_data['mag_std'], 
                  label=f'MET Class {met_class}', alpha=0.6)
    
    ax.set_xlabel('Mean Magnitude')
    ax.set_ylabel('Std Magnitude') 
    ax.set_title(f'Window Size: {window_size} samples')
    ax.legend()
    ax.grid(True)

# Plot comparison of smoothed vs raw magnitude sensitivity
if len(window_sizes) > 2:
    ax = axes[1, 1]
else:
    fig2, ax = plt.subplots(1, 1, figsize=(8, 6))

# Sample data for demonstration
sample_data = df[df['user'] == 1].head(200)
ax.plot(sample_data['magnitude'], label='Raw Magnitude', alpha=0.7)
ax.plot(sample_data['magnitude_smoothed'], label='Smoothed Magnitude', alpha=0.7)
ax.axhline(y=1.05, color='r', linestyle='--', label='Threshold (Heuristic)')
ax.set_xlabel('Time (samples)')
ax.set_ylabel('Acceleration Magnitude')
    ax.set_title('Raw vs Smoothed Magnitude - Reducing Noise from Sudden Movements')
ax.legend()
ax.grid(True)

plt.tight_layout()
plt.show()

# Calculate threshold crossing analysis
print("\n=== THRESHOLD CROSSING ANALYSIS ===")
threshold = 1.05  # Current heuristic threshold

for window_size in window_sizes:
    feature_df = feature_datasets[window_size]
    
    # Calculate how often magnitude crosses threshold vs sustained activity
    raw_crossings = (feature_df['mag_max'] > threshold).sum()
    sustained_crossings = (feature_df['mag_mean'] > threshold).sum()
    
    print(f"Window Size {window_size}:")
    print(f"  - Raw magnitude crossings: {raw_crossings}/{len(feature_df)} ({raw_crossings/len(feature_df)*100:.1f}%)")
    print(f"  - Sustained (mean) crossings: {sustained_crossings}/{len(feature_df)} ({sustained_crossings/len(feature_df)*100:.1f}%)")
    print(f"  - Reduction factor: {raw_crossings/max(sustained_crossings, 1):.1f}x")

In [ ]:
# Model Comparison and Selection
print("=== MODEL COMPARISON ===")

# Define models to compare
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'SVM': SVC(random_state=42, probability=True),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42, n_estimators=100),
    'Neural Network': MLPClassifier(random_state=42, max_iter=500)
}

# Prepare feature columns (exclude metadata)
feature_cols = ['mag_mean', 'mag_std', 'mag_min', 'mag_max', 'mag_range', 'mag_iqr',
                'mag_smooth_mean', 'mag_smooth_std',
                'x_mean', 'x_std', 'y_mean', 'y_std', 'z_mean', 'z_std',
                'xy_corr', 'xz_corr', 'yz_corr', 'x_zcr', 'y_zcr', 'z_zcr']

# Test each window size and model combination
results = {}

for window_size in window_sizes:
    print(f"\n--- Window Size: {window_size} samples ---")
    feature_df = feature_datasets[window_size]
    
    X = feature_df[feature_cols]
    y = feature_df['met_class']
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                        random_state=42, stratify=y)
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    window_results = {}
    
    for model_name, model in models.items():
        print(f"  Training {model_name}...")
        
        # Train model
        start_time = time.time()
        model.fit(X_train_scaled, y_train)
        training_time = time.time() - start_time
        
        # Test inference time (important for mobile)
        start_time = time.time()
        y_pred = model.predict(X_test_scaled)
        inference_time = (time.time() - start_time) / len(X_test) * 1000  # ms per prediction
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5)
        
        window_results[model_name] = {
            'accuracy': accuracy,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'training_time': training_time,
            'inference_time_ms': inference_time,
            'model': model,
            'scaler': scaler,
            'y_pred': y_pred,
            'y_test': y_test
        }
        
        print(f"    Accuracy: {accuracy:.4f} ± {cv_scores.std():.4f}")
        print(f"    Inference: {inference_time:.2f}ms per prediction")
    
    results[window_size] = window_results

print("\n=== SUMMARY RESULTS ===")
for window_size in window_sizes:
    print(f"\nWindow Size {window_size}:")
    for model_name in models.keys():
        res = results[window_size][model_name]
        print(f"  {model_name}: {res['accuracy']:.4f} (±{res['cv_std']:.4f}) - {res['inference_time_ms']:.2f}ms")

In [ ]:
# Hyperparameter Tuning for Best Models
print("=== HYPERPARAMETER TUNING ===")

# Identify best performing models across all window sizes
best_combinations = []
for window_size in window_sizes:
    for model_name in models.keys():
        result = results[window_size][model_name]
        best_combinations.append({
            'window_size': window_size,
            'model_name': model_name,
            'accuracy': result['accuracy'],
            'inference_time': result['inference_time_ms']
        })

# Sort by accuracy and filter by reasonable inference time (<50ms)
best_combinations = sorted(best_combinations, key=lambda x: x['accuracy'], reverse=True)
mobile_suitable = [x for x in best_combinations if x['inference_time'] < 50]

print("Top 3 mobile-suitable model combinations:")
for i, combo in enumerate(mobile_suitable[:3]):
    print(f"{i+1}. {combo['model_name']} (Window: {combo['window_size']}) - "
          f"Acc: {combo['accuracy']:.4f}, Time: {combo['inference_time']:.1f}ms")

# Tune the best model
best_combo = mobile_suitable[0]
best_window = best_combo['window_size']
best_model_name = best_combo['model_name']

print(f"\nTuning {best_model_name} with window size {best_window}...")

# Get data for best configuration
feature_df = feature_datasets[best_window]
X = feature_df[feature_cols]
y = feature_df['met_class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define parameter grids for tuning
param_grids = {
    'Random Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    },
    'Gradient Boosting': {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.1, 0.05, 0.01]
    },
    'SVM': {
        'C': [0.1, 1, 10],
        'gamma': ['scale', 'auto'],
        'kernel': ['rbf', 'linear']
    }
}

if best_model_name in param_grids:
    param_grid = param_grids[best_model_name]
    base_model = models[best_model_name]
    
    # Use TimeSeriesSplit for more realistic evaluation
    tscv = TimeSeriesSplit(n_splits=3)
    
    grid_search = GridSearchCV(base_model, param_grid, cv=tscv, 
                              scoring='accuracy', n_jobs=-1, verbose=1)
    
    print("Running grid search...")
    grid_search.fit(X_train_scaled, y_train)
    
    # Get best model
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    
    print(f"Best parameters: {best_params}")
    print(f"Best CV score: {grid_search.best_score_:.4f}")
    
    # Test tuned model
    y_pred_tuned = best_model.predict(X_test_scaled)
    tuned_accuracy = accuracy_score(y_test, y_pred_tuned)
    
    print(f"Test accuracy (tuned): {tuned_accuracy:.4f}")
    print(f"Improvement: {tuned_accuracy - best_combo['accuracy']:.4f}")
else:
    best_model = results[best_window][best_model_name]['model']
    best_params = "Default parameters"
    y_pred_tuned = results[best_window][best_model_name]['y_pred']
    tuned_accuracy = results[best_window][best_model_name]['accuracy']

In [ ]:
# Model Evaluation and Performance Analysis
print("=== DETAILED MODEL EVALUATION ===")

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_tuned, 
                          target_names=['Sedentary', 'Light', 'Moderate', 'Vigorous']))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_tuned)
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Confusion matrix heatmap
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Sedentary', 'Light', 'Moderate', 'Vigorous'],
            yticklabels=['Sedentary', 'Light', 'Moderate', 'Vigorous'],
            ax=axes[0])
axes[0].set_title(f'Confusion Matrix - {best_model_name}')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# Model comparison chart
comparison_data = []
for window_size in window_sizes:
    for model_name in models.keys():
        result = results[window_size][model_name]
        comparison_data.append({
            'Model': model_name,
            'Window Size': window_size,
            'Accuracy': result['accuracy'],
            'Inference Time (ms)': result['inference_time_ms']
        })

comp_df = pd.DataFrame(comparison_data)

# Accuracy vs Inference Time scatter plot
for model_name in models.keys():
    model_data = comp_df[comp_df['Model'] == model_name]
    axes[1].scatter(model_data['Inference Time (ms)'], model_data['Accuracy'], 
                   label=model_name, s=100, alpha=0.7)

axes[1].set_xlabel('Inference Time (ms)')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Model Performance: Accuracy vs Inference Time')
axes[1].legend()
axes[1].grid(True)
axes[1].axvline(x=50, color='r', linestyle='--', alpha=0.5, label='Mobile Limit (50ms)')

plt.tight_layout()
plt.show()

# Per-class accuracy analysis
class_accuracies = []
for i, class_name in enumerate(['Sedentary', 'Light', 'Moderate', 'Vigorous']):
    class_mask = (y_test == i)
    if class_mask.sum() > 0:
        class_acc = accuracy_score(y_test[class_mask], y_pred_tuned[class_mask])
        class_accuracies.append(class_acc)
        print(f"{class_name} accuracy: {class_acc:.4f}")

# Plot class-wise accuracy
plt.figure(figsize=(10, 6))
plt.bar(['Sedentary', 'Light', 'Moderate', 'Vigorous'], class_accuracies, 
        color=['blue', 'green', 'orange', 'red'], alpha=0.7)
plt.ylabel('Accuracy')
plt.title('Per-Class Accuracy')
plt.ylim(0, 1)
for i, acc in enumerate(class_accuracies):
    plt.text(i, acc + 0.02, f'{acc:.3f}', ha='center')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Feature Importance Analysis
print("=== FEATURE IMPORTANCE ANALYSIS ===")

if hasattr(best_model, 'feature_importances_'):
    # Tree-based model feature importance
    importances = best_model.feature_importances_
    feature_importance_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    # Plot feature importance
    plt.figure(figsize=(12, 8))
    top_features = feature_importance_df.head(15)
    plt.barh(range(len(top_features)), top_features['importance'], 
             color='steelblue', alpha=0.7)
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Feature Importance')
    plt.title(f'Top 15 Feature Importances - {best_model_name}')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("Top 10 most important features:")
    for i, row in feature_importance_df.head(10).iterrows():
        print(f"{row['feature']}: {row['importance']:.4f}")

elif hasattr(best_model, 'coef_'):
    # Linear model coefficients
    coefs = np.abs(best_model.coef_).mean(axis=0)  # Average across classes
    feature_importance_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': coefs
    }).sort_values('importance', ascending=False)
    
    print("Top 10 features by coefficient magnitude:")
    for i, row in feature_importance_df.head(10).iterrows():
        print(f"{row['feature']}: {row['importance']:.4f}")

# Correlation analysis between features and MET classes
print("\n=== FEATURE-TARGET CORRELATION ===")
feature_df = feature_datasets[best_window]
correlations = {}

for feature in feature_cols:
    if feature in feature_df.columns:
        corr = feature_df[feature].corr(feature_df['met_class'])
        if not np.isnan(corr):
            correlations[feature] = abs(corr)

# Sort by correlation strength
sorted_correlations = sorted(correlations.items(), key=lambda x: x[1], reverse=True)

print("Top 10 features by correlation with MET class:")
for feature, corr in sorted_correlations[:10]:
    print(f"{feature}: {corr:.4f}")

# Create correlation heatmap for top features
top_corr_features = [item[0] for item in sorted_correlations[:10]]
corr_data = feature_df[top_corr_features + ['met_class']].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_data, annot=True, cmap='coolwarm', center=0, 
            fmt='.3f', square=True)
plt.title('Feature Correlation Matrix (Top 10 Features)')
plt.tight_layout()
plt.show()

In [ ]:
# Real-time Prediction Pipeline Implementation  
print("=== REAL-TIME PREDICTION PIPELINE ===")

class METRealTimePipeline:
    """Complete pipeline for real-time MET prediction"""
    
    def __init__(self, model, scaler, feature_cols, window_size=50):
        self.model = model
        self.scaler = scaler
        self.feature_cols = feature_cols
        self.window_size = window_size
        self.data_buffer = []
        self.prediction_history = []
        
    def add_accelerometer_reading(self, x, y, z, timestamp=None):
        """Add new accelerometer reading to buffer"""
        magnitude = np.sqrt(x**2 + y**2 + z**2)
        self.data_buffer.append({
            'x': x, 'y': y, 'z': z, 
            'magnitude': magnitude,
            'timestamp': timestamp or time.time()
        })
        
        # Keep only the required window size
        if len(self.data_buffer) > self.window_size * 2:
            self.data_buffer = self.data_buffer[-self.window_size * 2:]
    
    def extract_features_from_buffer(self):
        """Extract features from current buffer"""
        if len(self.data_buffer) < self.window_size:
            return None
        
        # Use the most recent window
        window_data = self.data_buffer[-self.window_size:]
        df_window = pd.DataFrame(window_data)
        
        # Apply smoothing
        df_window['magnitude_smoothed'] = apply_smoothing_filter(df_window['magnitude'].values)
        
        # Extract features (same as training)
        features = {
            'mag_mean': df_window['magnitude'].mean(),
            'mag_std': df_window['magnitude'].std(),
            'mag_min': df_window['magnitude'].min(),
            'mag_max': df_window['magnitude'].max(),
            'mag_range': df_window['magnitude'].max() - df_window['magnitude'].min(),
            'mag_iqr': df_window['magnitude'].quantile(0.75) - df_window['magnitude'].quantile(0.25),
            'mag_smooth_mean': df_window['magnitude_smoothed'].mean(),
            'mag_smooth_std': df_window['magnitude_smoothed'].std(),
            'x_mean': df_window['x'].mean(),
            'x_std': df_window['x'].std(),
            'y_mean': df_window['y'].mean(), 
            'y_std': df_window['y'].std(),
            'z_mean': df_window['z'].mean(),
            'z_std': df_window['z'].std(),
            'xy_corr': df_window['x'].corr(df_window['y']),
            'xz_corr': df_window['x'].corr(df_window['z']),
            'yz_corr': df_window['y'].corr(df_window['z']),
            'x_zcr': np.sum(np.diff(np.signbit(df_window['x']))) / len(df_window),
            'y_zcr': np.sum(np.diff(np.signbit(df_window['y']))) / len(df_window),
            'z_zcr': np.sum(np.diff(np.signbit(df_window['z']))) / len(df_window),
        }
        
        # Handle NaN correlations
        for key in ['xy_corr', 'xz_corr', 'yz_corr']:
            if np.isnan(features[key]):
                features[key] = 0.0
        
        return features
    
    def predict_met_class(self):
        """Predict MET class from current buffer"""
        features = self.extract_features_from_buffer()
        if features is None:
            return None, 0.0
        
        # Convert to feature array
        feature_array = np.array([features[col] for col in self.feature_cols]).reshape(1, -1)
        
        # Scale and predict
        feature_array_scaled = self.scaler.transform(feature_array)
        prediction = self.model.predict(feature_array_scaled)[0]
        
        # Get confidence if available
        if hasattr(self.model, 'predict_proba'):
            probabilities = self.model.predict_proba(feature_array_scaled)[0]
            confidence = probabilities.max()
        else:
            confidence = 1.0
        
        # Store prediction
        self.prediction_history.append({
            'timestamp': time.time(),
            'prediction': prediction,
            'confidence': confidence
        })
        
        return prediction, confidence
    
    def get_smoothed_prediction(self, smoothing_window=3):
        """Get smoothed prediction to avoid rapid class changes"""
        if len(self.prediction_history) < smoothing_window:
            return self.prediction_history[-1]['prediction'] if self.prediction_history else 0
        
        recent_predictions = [p['prediction'] for p in self.prediction_history[-smoothing_window:]]
        # Use mode (most common prediction)
        return max(set(recent_predictions), key=recent_predictions.count)

# Create pipeline instance
pipeline = METRealTimePipeline(best_model, scaler, feature_cols, best_window)

print(f"Pipeline created with:")
print(f"  - Model: {best_model_name}")
print(f"  - Window size: {best_window} samples")
print(f"  - Features: {len(feature_cols)} features")

# Test pipeline with sample data
print("\n=== PIPELINE TESTING ===")
test_user_data = feature_datasets[best_window][feature_datasets[best_window]['user'] == 1].head(100)

print("Testing pipeline with sample accelerometer data...")
predictions = []
confidences = []

# Simulate real-time data input
for _, row in test_user_data.iterrows():
    # Simulate adding accelerometer readings (we need to reverse engineer from features)
    # For testing, we'll use synthetic data with known patterns
    if row['met_class'] == 0:  # Sedentary
        x, y, z = np.random.normal(0, 0.5), np.random.normal(0, 0.5), np.random.normal(9.8, 0.5)
    elif row['met_class'] == 1:  # Light
        x, y, z = np.random.normal(0, 1.5), np.random.normal(0, 1.5), np.random.normal(9.8, 1.5)
    elif row['met_class'] == 2:  # Moderate  
        x, y, z = np.random.normal(0, 3.0), np.random.normal(0, 3.0), np.random.normal(9.8, 3.0)
    else:  # Vigorous
        x, y, z = np.random.normal(0, 5.0), np.random.normal(0, 5.0), np.random.normal(9.8, 5.0)
    
    pipeline.add_accelerometer_reading(x, y, z)
    
    # Try to get prediction
    pred, conf = pipeline.predict_met_class()
    if pred is not None:
        predictions.append(pred)
        confidences.append(conf)

if predictions:
    print(f"Pipeline test completed:")
    print(f"  - Made {len(predictions)} predictions")
    print(f"  - Average confidence: {np.mean(confidences):.3f}")
    print(f"  - Prediction distribution: {np.bincount(predictions)}")

# Test inference timing
print("\n=== PERFORMANCE TIMING ===")
timing_tests = []
for _ in range(100):
    start_time = time.time()
    pred, conf = pipeline.predict_met_class()
    if pred is not None:
        timing_tests.append((time.time() - start_time) * 1000)

if timing_tests:
    print(f"Inference timing (100 tests):")
    print(f"  - Mean: {np.mean(timing_tests):.2f}ms")
    print(f"  - Std: {np.std(timing_tests):.2f}ms") 
    print(f"  - 95th percentile: {np.percentile(timing_tests, 95):.2f}ms")
    print(f"  - Max: {np.max(timing_tests):.2f}ms")

In [ ]:
# Model Export for Mobile Integration
print("=== MODEL EXPORT FOR MOBILE ===")

import pickle
import json
from datetime import datetime

# Create models directory
models_dir = "../models"
os.makedirs(models_dir, exist_ok=True)
os.makedirs(f"{models_dir}/mobile", exist_ok=True)

# Save the complete model pipeline
model_data = {
    'model': best_model,
    'scaler': scaler,
    'feature_cols': feature_cols,
    'window_size': best_window,
    'model_name': best_model_name,
    'accuracy': tuned_accuracy,
    'parameters': best_params if 'best_params' in locals() else 'default'
}

# Save as pickle for Python usage
model_file = f"{models_dir}/met_model_final.pkl"
with open(model_file, 'wb') as f:
    pickle.dump(model_data, f)

print(f"Model saved to: {model_file}")

# Save model metadata as JSON
metadata = {
    'model_name': best_model_name,
    'window_size': best_window,
    'feature_columns': feature_cols,
    'accuracy': tuned_accuracy,
    'training_date': datetime.now().isoformat(),
    'parameters': str(best_params) if 'best_params' in locals() else 'default',
    'target_classes': ['Sedentary', 'Light', 'Moderate', 'Vigorous'],
    'inference_time_ms': np.mean(timing_tests) if timing_tests else 'not_measured'
}

metadata_file = f"{models_dir}/model_metadata.json"
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"Metadata saved to: {metadata_file}")

# For iOS integration, save simplified model parameters
if best_model_name == 'Random Forest':
    # Extract tree structure for mobile implementation
    mobile_params = {
        'model_type': 'random_forest',
        'n_estimators': best_model.n_estimators,
        'max_depth': best_model.max_depth,
        'feature_importances': best_model.feature_importances_.tolist(),
        'feature_names': feature_cols
    }
elif best_model_name == 'SVM':
    mobile_params = {
        'model_type': 'svm',
        'support_vectors': best_model.support_vectors_.tolist() if hasattr(best_model, 'support_vectors_') else [],
        'dual_coef': best_model.dual_coef_.tolist() if hasattr(best_model, 'dual_coef_') else [],
        'intercept': best_model.intercept_.tolist() if hasattr(best_model, 'intercept_') else []
    }
else:
    mobile_params = {
        'model_type': best_model_name.lower().replace(' ', '_'),
        'note': 'Complex model - consider simplification for mobile'
    }

# Save scaler parameters for mobile
scaler_params = {
    'mean': scaler.mean_.tolist(),
    'scale': scaler.scale_.tolist(),
    'feature_names': feature_cols
}

mobile_model_file = f"{models_dir}/mobile/model_params.json"
mobile_scaler_file = f"{models_dir}/mobile/scaler_params.json"

with open(mobile_model_file, 'w') as f:
    json.dump(mobile_params, f, indent=2)

with open(mobile_scaler_file, 'w') as f:
    json.dump(scaler_params, f, indent=2)

print(f"Mobile model parameters saved to: {mobile_model_file}")
print(f"Mobile scaler parameters saved to: {mobile_scaler_file}")

# Create implementation recommendations
recommendations = f"""
=== IMPLEMENTATION RECOMMENDATIONS ===

1. OPTIMAL CONFIGURATION:
   - Model: {best_model_name}
   - Window Size: {best_window} samples (~{best_window/20:.1f} seconds at 20Hz)
   - Accuracy: {tuned_accuracy:.4f}
   - Inference Time: {np.mean(timing_tests):.1f}ms (mobile suitable)

2. TIME WINDOW INSIGHTS:
   - Momentary acceleration spikes should NOT trigger immediate class changes
   - Use {best_window} sample windows with 50% overlap for stability
   - Apply smoothing filter to reduce noise from sudden movements
   - Consider majority voting over 3 consecutive predictions

3. MOBILE IMPLEMENTATION:
   - Buffer {best_window} accelerometer readings
   - Extract {len(feature_cols)} features per window
   - Apply standardization using saved scaler parameters
   - Predict using {best_model_name} model
   - Use prediction smoothing to avoid rapid class switching

4. FEATURE PRIORITIES:
   Top 5 most important features for mobile optimization:
"""

if 'feature_importance_df' in locals():
    for i, row in feature_importance_df.head(5).iterrows():
        recommendations += f"   - {row['feature']}: {row['importance']:.4f}\n"

print(recommendations)

# Save recommendations
with open(f"{models_dir}/implementation_recommendations.txt", 'w') as f:
    f.write(recommendations)

print(f"\n=== MODEL SELECTION COMPLETE ===")
print(f"Best model ready for iOS integration!")
print(f"All files saved in {models_dir}/ directory")